In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install -q kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.8 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-4b65jjng
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-4b65jjng
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [ ]:
from PIL import Image
import requests
import torch
# import clip
from transformers import CLIPProcessor, CLIPModel
from google.colab import userdata
import os
from tqdm import tqdm
import torchvision.transforms as transforms

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
!kaggle datasets download -d ubitquitin/geolocation-geoguessr-images-50k
!unzip -q "geolocation-geoguessr-images-50k.zip" -d "/content/dataset"
data_dir = '/content/dataset/compressed_dataset'
countries = os.listdir(data_dir)
model = CLIPModel.from_pretrained("geolocal/StreetCLIP")
processor = CLIPProcessor.from_pretrained("geolocal/StreetCLIP")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Model device:", next(model.parameters()).device)

Dataset URL: https://www.kaggle.com/datasets/ubitquitin/geolocation-geoguessr-images-50k
License(s): unknown
100% 6.66G/6.66G [05:30<00:00, 22.1MB/s]
100% 6.66G/6.66G [05:30<00:00, 21.6MB/s]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.95k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

Model device: cuda:0


In [ ]:
text_choices = []
for c in countries:
  text_choices.append('A street view image from ' + c)

accuracies = []
for country in countries:
  image_urls = os.listdir(data_dir + country)
  total = len(image_urls)
  wrong = 0
  for img_url in tqdm(image_urls):
    image = Image.open(data_dir + country + '/' + img_url)
    inputs = processor(text=text_choices, images=image, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)
    probs = outputs.logits_per_image.softmax(dim=1)[0]
    # del inputs, outputs, probs
    # torch.cuda.empty_cache()
  accuracies.append(1-wrong/total)

 41%|████▏     | 572/1382 [03:03<04:20,  3.11it/s]


KeyboardInterrupt: 

In [ ]:
text_choices = []
for c in countries:
  text_choices.append('A street view image from ' + c)

all_image_paths = []
true_classes = []
for country in countries:
  img_names = os.listdir(os.path.join(data_dir, country))
  for img_name in img_names:
    all_image_paths.append(os.path.join(data_dir, country, img_name))
    true_classes.append(country)

transform = transforms.Compose([
    transforms.Resize((331, 768)),
    transforms.ToTensor(),  # This will scale pixels between 0 and 1
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

errors = {country: 0 for country in countries}
batch_size = 3
for i in tqdm(range(0, len(all_image_paths), batch_size)):
  if i + batch_size >= len(all_image_paths):
    batch_paths = all_image_paths[i:]
    batch_classes = true_classes[i:]
  else:
    batch_paths = all_image_paths[i:i + batch_size]
    batch_classes = true_classes[i:i + batch_size]
  images = [transform(Image.open(path).convert("RGB")) for path in batch_paths]
  images_tensor = torch.stack(images).to(device)
  inputs = processor(text=text_choices, images=images_tensor, return_tensors="pt", padding=True, do_rescale=True)
  outputs = model(**inputs.to(device)).logits_per_image

  predicted_indices = torch.argmax(outputs.cpu().softmax(dim=1), dim=1)
  for j, pred_index in enumerate(predicted_indices.tolist()):
    if batch_classes[j] != countries[pred_index]:
      errors[batch_classes[j]] += 1
  # del inputs, images_tensor, outputs #, probs
  # torch.cuda.empty_cache()

  0%|          | 1/16666 [00:00<4:05:06,  1.13it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 7.06 MiB is free. Process 24754 has 14.74 GiB memory in use. Of the allocated memory 14.07 GiB is allocated by PyTorch, and 544.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# del images_tensor
# del inputs, outputs, images_tensor
# torch.cuda.empty_cache()
# import gc
# gc.collect()
# torch.cuda.empty_cache()
# !nvidia-smi
# model.to('cpu')